# This notebook has been productionized in .\tensor-hero\Preprocessing\chart_functions.py

see function chart2dict()

# 1. Get data from the .chart file

In [1]:
from pathlib import Path

In [2]:
# Let's start by opening a .chart file and reading it

# Define path to chart
unprocessed_data_path = Path(r'X:\Training Data\Unprocessed')
chartpath = r'X:\Training Data\Unprocessed\Anti Hero\Noel Pix - Die Legende (m9)\notes.chart'
# chartpath = unprocessed_data_path / 'Anti Hero' / '01. Brave This Storm (Miscellany)' / 'notes.chart'

# d = str(Path().resolve().parent)
# chartpath = d+'\\tensor-hero\Training Data\Audioslave - Exploder (Chezy)\\notes.chart'

# Read chart into array
with open(chartpath, 'r', encoding='utf-8') as file:
    raw_chart = file.readlines()
file.close()

# Strip lines of \n character
for i in range(len(raw_chart)):
    raw_chart[i] = raw_chart[i].replace('\n', '')

In [3]:
# Create lists to hold sections of of the chart file
song = []
synctrack = []
expertsingle = []

# Parse chart file, populating lists
i = 0
for data in raw_chart:
    if '[Song]' in data:
        i = 1
    elif data == '[SyncTrack]':
        i = 2
    elif data in ['[Events]', '[EasySingle]', '[MediumSingle]','[HardSingle]']:
        i = 3
    elif data == '[ExpertSingle]':
        i = 4
    
    if data in ['{', '}', '[Song]', 'ï»¿[Song]', '[SyncTrack]', '[Events]', '[ExpertSingle]']:
        continue
    if data[0] == ' ':
        if i == 1:
            song.append(data[2:])
        elif i == 2:
            synctrack.append(data[2:])
        elif i == 3:
            continue
        elif i == 4:
            expertsingle.append(data[2:])
    else:
        if i == 1:
            song.append(data[0:].strip('\t'))
        elif i == 2:
            synctrack.append(data[0:].strip('\t'))
        elif i == 3:
            continue
        elif i == 4:
            expertsingle.append(data[0:].strip('\t'))

time_signatures = {'tick' : [],
                    'TS' : []}
BPMs = {'tick' : [],
        'BPM' : []}

for event in synctrack:
    line = event.split(' ')
    if line[2] == 'TS':
        time_signatures['tick'].append(int(line[0]))
        time_signatures['TS'].append(int(line[3]))
    elif line[2] == 'R':
        raise NameError('Error - Resolution changes during this song')
    else:
        try:
            BPMs['tick'].append(int(line[0]))
            BPMs['BPM'].append(int(line[3]))
        except:
            # print(line)
            # print(expertsingle)
            # print(time_signatures)
            raise NameError('Chart file may have improper indentation')




In [7]:
print(song)

['Offset = 0', 'Resolution = 480', 'Player2 = bass', 'Difficulty = 0', 'PreviewStart = 0', 'PreviewEnd = 0', 'Genre = "rock"', 'MediaType = "cd"', 'GuitarStream = "guitar.ogg"']


In [8]:
# Parse the 'synctrack' section of the .chart file for timing information
time_signatures = {'tick' : [],
                   'TS' : []}
BPMs = {'tick' : [],
        'BPM' : []}

for event in synctrack:
    line = event.split(' ')
    print(line)
    if line[2] == 'TS':
        time_signatures['tick'].append(int(line[0]))
        time_signatures['TS'].append(int(line[3]))
    elif line[2] == 'R':
        raise NameError('Error - Resolution changes during this song')
    else:
        BPMs['tick'].append(int(line[0]))
        BPMs['BPM'].append(int(line[3]))

['0', '=', 'TS', '4']
['0', '=', 'B', '102040']
['1920', '=', 'B', '166232']
['7680', '=', 'B', '165451']
['26880', '=', 'B', '170000']


In [9]:
# Parse the 'expertsingle' section of the .chart file for note information
notes = {'tick' : [],        # What tick the note is at
         'N_S' : [],         # Whether it is a note (N) or star power (S)
         'note' : [],        # What the note is 
         'duration': []}     # tick duration of the note or star power

for note in expertsingle:
    line = note.split(' ')
    if len(line) < 2:
        break
    if line[2] == 'E':       # skip the lines that mark events
        continue
    else:
        if "=" in line:
            notes['tick'].append(int(line[0]))
            notes['N_S'].append(line[2])
            notes['note'].append(line[3])
            notes['duration'].append(int(line[4]))
        else:
            notes['tick'].append(int(line[0]))
            notes['N_S'].append(line[1])
            notes['note'].append(line[2])
            notes['duration'].append(int(line[3]))


In [10]:
# Parse the 'song' section of the .chart file to get relevent chart information
song_metadata = {'Name' : '',
                 'Artist' : '',
                 'Charter' : '',
                 'Offset' : '',
                 'Resolution' : '',
                 'Genre' : '',
                 'MediaType' : '',
                 'MusicStream' : ''}

for data in song:
    line = data.split(' ')
    if line[0] in song_metadata.keys():
        song_metadata[line[0]] = line[-1]
song_metadata['Offset'] = int(float(song_metadata['Offset']))
song_metadata['Resolution'] = int(float(song_metadata['Resolution']))

# 2. Convert ticks, resolution, and BPM to fit system requirements

Each tick must correspond to 10ms of time with BPM = 120. 

The tensors each correspond to a 10ms window of time. 

This is why it is useful to change the ticks to represent 10ms each.

## Tick Conversion Formula
We'll say each 'bin' of a particular BPM and time signature corresponds to a section, $n$, of the song with $N$ sections total.

To get the rate the ticks, $R$, we use the following formula:

$R \frac{seconds}{tick} = resolution^{-1} \frac{beats}{tick} *  BPM^{-1} \frac{minutes}{beat} * 60 \frac{seconds}{minute}$

$R = \frac{60}{res * BPM}$

We want the tick rate to be, $R = .01 \frac{seconds}{tick}$

To achieve this, we must adjust $BPM$

If $res = 192$, $BPM_{new} = 31.25$

If $res = 480$, $BPM_{new} = 12.5$

Generally, $BPM_{new} = \frac{60}{res \cdot R}$

Let $X$ be the conversion factor.

$X = \frac{BPM_{new}}{BPM}$

Multiplying all points by this conversion factor, doing some fancy shifting, then setting BPM = 31250 will produce the correct result.

In [11]:
# Split the song into bins corresponding to particular time signatures and BPMs

# First, assemble some lists from the preprocessing step
note_keys = list(zip(notes['tick'], notes['N_S'], 
                notes['note'], notes['duration']))                    # (tick, 'N_S', note)
TS_events = list(zip(time_signatures['tick'], time_signatures['TS']))  # (tick, TS)
BPM_events = list(zip(BPMs['tick'], BPMs['BPM']))                      # (tick, BPM)

# Append None at the end of these lists so the loop knows where to stop
TS_events.append(None)
BPM_events.append(None)

# Loop through all the notes in the song
TS_index = 0
BPM_index = 0

cur_TS = TS_events[TS_index]                # Current time signature
cur_BPM = BPM_events[BPM_index]             # Current BPM
next_TS = None                              # Next time signature
next_BPM = None                             # Next BPM
if len(TS_events) > 1:
    next_TS = TS_events[TS_index + 1]
if len(BPM_events) > 1:
    next_BPM = BPM_events[BPM_index + 1]


# bins['TS'][0] corresponds to the time signature of bin 0
# bins['notes'] is a list of lists of notes in each bin
bins = {
    'TS' : [],              # time signature
    'BPM' : [],             # BPM
    'shift_tick' : [],      # The first tick where the TS / BPM combo starts
    'notes' : [[]],         # The notes in the bin
}

# Append the first element of each array before looping
event_index = 0     # Counts how many times either BPM or TS change
bins['TS'].append(cur_TS[1])
bins['BPM'].append(cur_BPM[1])
bins['shift_tick'].append(cur_BPM[0])
bins['notes'][event_index].append(note_keys[0])

# Initialize ticks
cur_TS_tick = cur_TS[0]
if next_TS != None:
    next_TS_tick = next_TS[0]
else:
    next_TS_tick = None
cur_BPM_tick = cur_BPM[0]
if next_BPM != None:
    next_BPM_tick = next_BPM[0]
else:
    next_BPM_tick = None

for i in range(1, len(note_keys)):
    if next_BPM_tick == None and next_TS_tick == None:     # If in the last bin
        bins['notes'][-1].append(note_keys[i])             # Add notes until there are no more to add
        continue
    
    if next_TS_tick != None:                        # If there is a time signature change in the future
        if note_keys[i][0] >= next_TS_tick:         # If the current note is past that change                            
            if next_BPM_tick != None:                   # If there is a BPM change in the future
                if note_keys[i][0] >= next_BPM_tick:    # If the current note is past that change
                    TS_index += 1                       # Update time signature and BPM, they changed at the same time
                    cur_TS = TS_events[TS_index]
                    cur_TS_tick = cur_TS[0]
                    next_TS = TS_events[TS_index + 1]
                    if next_TS != None:
                        next_TS_tick = next_TS[0]
                    else:
                        next_TS_tick = None

                    BPM_index += 1
                    cur_BPM = BPM_events[BPM_index]
                    cur_BPM_tick = cur_BPM[0]
                    next_BPM = BPM_events[BPM_index + 1]
                    if next_BPM != None:
                        next_BPM_tick = next_BPM[0]
                    else:
                        next_BPM_tick = None

                    bins['TS'].append(cur_TS[1])
                    bins['BPM'].append(cur_BPM[1])
                    bins['shift_tick'].append(min(cur_TS[0], cur_BPM[0]))
                    bins['notes'].append([])
                    bins['notes'][-1].append(note_keys[i])
                    continue

                else:                                   # If the time signature changed but the BPM didn't
                    TS_index += 1                       # Update the time signature, but not the BPM
                    cur_TS = TS_events[TS_index]
                    cur_TS_tick = cur_TS[0]
                    next_TS = TS_events[TS_index + 1]
                    if next_TS != None:
                        next_TS_tick = next_TS[0]
                    else:
                        next_TS_tick = None

                    bins['TS'].append(cur_TS[1])
                    bins['BPM'].append(cur_BPM[1])
                    bins['shift_tick'].append(min(cur_TS[0], cur_BPM[0]))
                    bins['notes'].append([])
                    bins['notes'][-1].append(note_keys[i])
                    continue

            else:                               # If the next BPM tick = None but the note tick is past the time signature
                TS_index += 1                   # Update the time signature, but not the BPM
                cur_TS = TS_events[TS_index]
                cur_TS_tick = cur_TS[0]
                next_TS = TS_events[TS_index + 1]
                if next_TS != None:
                    next_TS_tick = next_TS[0]
                else:
                    next_TS_tick = None       

                bins['TS'].append(cur_TS[1])
                bins['BPM'].append(cur_BPM[1])
                bins['shift_tick'].append(cur_TS[0])
                bins['notes'].append([])
                bins['notes'][-1].append(note_keys[i])
                continue

        else:  # If there is a time signature change in the future but the note is not past it
            if next_BPM_tick != None:                   # If there is a BPM change in the future
                if note_keys[i][0] >= next_BPM_tick:    # If the note is past that BPM change    
                    BPM_index += 1                      # Update the BPM but not the time signature
                    cur_BPM = BPM_events[BPM_index]
                    cur_BPM_tick = cur_BPM[0]
                    next_BPM = BPM_events[BPM_index + 1]
                    if next_BPM != None:
                        next_BPM_tick = next_BPM[0]
                    else:
                        next_BPM_tick = None

                    bins['TS'].append(cur_TS[1])
                    bins['BPM'].append(cur_BPM[1])
                    bins['shift_tick'].append(cur_BPM[0])
                    bins['notes'].append([])
                    bins['notes'][-1].append(note_keys[i])
                    continue

                else:  # If the time signature did not change and the BPM also did not change
                    bins['notes'][-1].append(note_keys[i])  # Add note and continue
                    continue

    #-------------------------------------------------------------------------------------------------------#
    # The second half of the ifzilla:
    # If there is not a time signature change in the future

    else:                        # If there is NOT a time signature change in the future                              
        if next_BPM_tick != None:                   # If there is a BPM change in the future
            if note_keys[i][0] >= next_BPM_tick:    # If the current note is past that change
                BPM_index += 1                      # Update the BPM
                cur_BPM = BPM_events[BPM_index]
                cur_BPM_tick = cur_BPM[0]
                next_BPM = BPM_events[BPM_index + 1]
                if next_BPM != None:
                    next_BPM_tick = next_BPM[0]
                else:
                    next_BPM_tick = None

                bins['TS'].append(cur_TS[1])
                bins['BPM'].append(cur_BPM[1])
                bins['shift_tick'].append(cur_BPM[0])
                bins['notes'].append([])
                bins['notes'][-1].append(note_keys[i])
                continue

            else:  # If the current note is not past the BPM change
                bins['notes'][-1].append(note_keys[i])  # Add note and continue
                continue

        else:                               # If the next BPM tick = None and the next TS tick = None
                                            # Then the if statement at the beginning of this thing should have fired off
            raise NameError('Error: Tick Conversion Failure')

In [23]:
print(next_TS_tick)

None


In [18]:
bins['X'] = []                # Tick conversion factor
bins['sync_tick'] = []        # The tick value that the first note in 'notes' should have
res = song_metadata['Resolution'] # resolution
BPM_new = int(60000 / (res*0.01))
print(BPM_new)
# Populate 'X' and 'sync_tick' field of bins
for i in range(len(bins['shift_tick'])):
    bins['X'].append(BPM_new / bins['BPM'][i])  # 31250 = 31.25 beats/minute * 1000, this BPM corresponds to 10ms per tick

    if i == 0:
        bins['sync_tick'].append(0)
    else:
        bins['sync_tick'].append(round(bins['notes'][i][0][0] * bins['X'][i-1]))


12500


In [20]:
# Sanity check
print(bins['sync_tick'])
print(bins['shift_tick'])
print(bins['X'])

[0, 235, 578, 2031]
[0, 1920, 7680, 26880]
[0.12250098000784006, 0.07519611145868425, 0.07555106950093986, 0.07352941176470588]


In [21]:
# Create array in bins for 'shift'
bins['shift'] = [0]  # Don't shift the first bin

# Convert note ticks to using conversion factor X
for i in range(len(bins['shift_tick'])):
    for j in range(len(bins['notes'][i])):
        bins['notes'][i][j] = list(bins['notes'][i][j])  # Convert to list
        
        # Construct shift length
        if j == 0 and i != 0:
            bins['shift'].append(round((bins['shift_tick'][i] * bins['X'][i])) - (round((bins['shift_tick'][i] * bins['X'][i-1])) - bins['shift'][i-1]))

        bins['notes'][i][j][0] = round(bins['notes'][i][j][0]*bins['X'][i])  # Scale tick mark
        bins['notes'][i][j][3] = round(bins['notes'][i][j][3]*bins['X'][i])  # Scale duration
        bins['notes'][i][j][0] -= bins['shift'][i]                           # Shift

In [22]:
# Convert back to dictionary format
shifted_notes = {'tick' : [],        # What tick the note is at
                  'N_S' : [],         # Whether it is a note (N) or star power (S)
                  'note' : [],        # What the note is 
                  'duration': []}     # tick duration of the note or star power

last_tick = bins['notes'][-1][-1][0]

for T_bin in bins['notes']:
    for n in T_bin:
        shifted_notes['tick'].append(n[0])
        shifted_notes['N_S'].append(n[1])
        shifted_notes['note'].append(n[2])
        shifted_notes['duration'].append(n[3])

print(shifted_notes)

{'tick': [235, 235, 253, 253, 271, 271, 290, 290, 308, 308, 326, 326, 344, 344, 362, 362, 380, 380, 398, 398, 416, 416, 434, 434, 452, 452, 470, 470, 488, 488, 506, 506, 524, 524, 542, 542, 560, 560, 578, 578, 596, 596, 614, 614, 632, 632, 650, 650, 669, 669, 687, 687, 705, 705, 724, 724, 742, 742, 760, 760, 778, 778, 796, 796, 814, 814, 832, 832, 851, 851, 869, 869, 887, 887, 905, 905, 923, 923, 941, 941, 959, 959, 977, 977, 996, 996, 1014, 1014, 1032, 1032, 1050, 1050, 1068, 1068, 1086, 1086, 1104, 1104, 1104, 1123, 1123, 1141, 1141, 1159, 1159, 1177, 1177, 1195, 1195, 1213, 1213, 1231, 1231, 1249, 1249, 1268, 1268, 1286, 1286, 1304, 1304, 1322, 1322, 1340, 1340, 1358, 1358, 1376, 1376, 1395, 1395, 1413, 1413, 1431, 1431, 1449, 1449, 1467, 1467, 1485, 1485, 1503, 1503, 1521, 1521, 1540, 1540, 1558, 1558, 1576, 1576, 1594, 1594, 1612, 1612, 1630, 1630, 1648, 1648, 1667, 1667, 1685, 1685, 1703, 1703, 1721, 1721, 1739, 1739, 1757, 1757, 1775, 1775, 1793, 1793, 1811, 1811, 1830, 1830, 18

In [27]:
print(bins['notes'].shape)

AttributeError: 'list' object has no attribute 'shape'

### Now we have all the components we need to write the new file

In [45]:
f = open('new song.chart', 'w')

# populate '[Song]' portion of file
f.writelines(['[Song]\n', '{\n'])
for k, v in song_metadata.items():
    f.writelines('  ' + k + ' = ' + str(v) + '\n')
f.writelines('}\n')

# Populate '[SyncTrack]' portion of file, skip [Events]
f.writelines('[SyncTrack]\n{\n  0 = TS 1\n  0 = B 31250\n}\n[Events]\n{\n}\n')

# Populate notes in '[ExpertSingle]'
f.writelines('[ExpertSingle]\n{\n')
for i in range(len(bins['shift_tick'])):
    for j in range(len(bins['notes'][i])):
        f.writelines('  ' + str(bins['notes'][i][j][0]) + ' = ' +
                     str(bins['notes'][i][j][1]) + ' ' + 
                     str(bins['notes'][i][j][2]) + ' ' +
                     str(bins['notes'][i][j][3]) + '\n')
f.writelines('}')
f.close()

### It works! Note that it will be advantageous to convert songs back to a more reasonable BPM like 120 after processing them. This is because everything turns into a hammer on with a BPM as slow as 31.25

In [74]:
# Test chart2dict to evaluate output
sys.path.insert(1, str(Path().resolve().parent) + r'\Preprocessing')
from chart_functions import chart2dict

notes, _, _, _ = chart2dict(chartpath)

print(notes['N_S'][i])

N
